In [2]:
import cv2
from deepface import DeepFace
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import threading

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

webcam_active = False

In [3]:
def analyze_webcam():
    global webcam_active
    webcam_active = True

    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    while webcam_active:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        result = None
        try:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        except Exception as e:
            print(f"DeepFace analysis error: {e}")

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        if result is not None:
            dominant_emotion = result[0]['dominant_emotion']
            cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_4)

        cv2.imshow('Webcam Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [4]:
def start_webcam_thread():
    webcam_thread = threading.Thread(target=analyze_webcam)
    webcam_thread.start()

def stop_webcam():
    global webcam_active
    webcam_active = False

In [5]:
def resize_image(image, max_width, max_height):
    # Calculate the ratio of the width and height
    width_ratio = max_width / image.width
    height_ratio = max_height / image.height
    # Use the smallest ratio to resize the image
    ratio = min(width_ratio, height_ratio)
    new_width = int(image.width * ratio)
    new_height = int(image.height * ratio)
    return image.resize((new_width, new_height), Image.LANCZOS)


In [6]:
def analyze_file():
    global panel, emotion_label
    file_path = filedialog.askopenfilename()
    if not file_path:
        return

    if panel is not None:
        panel.pack_forget()
    if emotion_label is not None:
        emotion_label.config(text="")

    if file_path.endswith(('.mp4', '.avi', '.mov')):
        cap = cv2.VideoCapture(file_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            result = None
            try:
                result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            except Exception as e:
                print(f"DeepFace analysis error: {e}")

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(gray, 1.1, 4)

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            if result is not None:
                dominant_emotion = result[0]['dominant_emotion']
                cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_4)

            cv2.imshow('Video File', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    else:
        frame = cv2.imread(file_path)
        result = None
        try:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        except Exception as e:
            print(f"DeepFace analysis error: {e}")

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        if result is not None:
            dominant_emotion = result[0]['dominant_emotion']
            emotion_label.config(text=f"Dominant Emotion: {dominant_emotion}")

        # Resize the image to fit the GUI window
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = resize_image(img, 800, 600)  # Resize to fit window
        img_tk = ImageTk.PhotoImage(image=img)

        panel.config(image=img_tk)
        panel.image = img_tk
        panel.pack()

In [7]:
root = tk.Tk()
root.title("Emotion Analyzer")
root.geometry("1000x800")
root.configure(bg="#f0f0f0")


style = ttk.Style()
style.configure('TButton', font=('Helvetica', 12), padding=10)

panel = tk.Label(root, bg="#f0f0f0")
panel.pack(pady=10)

emotion_label = tk.Label(root, text="", font=("Helvetica", 16), bg="#f0f0f0")
emotion_label.pack(pady=10)


button_frame = tk.Frame(root, bg="#f0f0f0")
button_frame.pack(pady=20)


In [ ]:
btn_webcam = ttk.Button(button_frame, text="Analyze from Webcam", command=start_webcam_thread)
btn_webcam.grid(row=0, column=0, padx=10)

btn_stop_webcam = ttk.Button(button_frame, text="Stop Webcam", command=stop_webcam)
btn_stop_webcam.grid(row=0, column=1, padx=10)

btn_file = ttk.Button(button_frame, text="Analyze from File", command=analyze_file)
btn_file.grid(row=0, column=2, padx=10)

import threading

root.mainloop()